In [1]:
import warnings
warnings.filterwarnings('ignore')

Meteorological satellite data is used by many for getting an idea of what the earth looks like at any given time. Creating images or other visualizations of satellite data can quickly become complex with the many operations and concepts involved. In the rest of this article I'll describe the basics concepts involved in meteorlogical satellite imager data and show some examples of how tweaking a few parameters can change what you see in the final image. I've simplified certain parts for two main reasons:

1. So people new to working with this data can understand them 
2. I don't know everything

I made this document as a [Jupyter Notebook](http://jupyter.org/) to make it easier to show text, code, and images together. The code shown is the actual python or bash code run to generate the images shown. In some cases I have used additional commands to reduce the size/quality of the generated images so they can be shown better in a browser.

The code shown here uses the [SatPy](https://satpy.readthedocs.io/en/latest/) library that I work on and other python packages created by the [PyTroll](http://pytroll.github.io/) team. For installation instructions see the SatPy documentation. I highly recommend using [conda](https://conda.io/) with the [conda-forge](https://conda-forge.org/) channel added to install SatPy in a Python 3.6+ environment. Note that some of the images may involve small customizations that are not the default in SatPy, where I do this I've tried to mention it. The data files used in these examples were received and processed at the SSEC at the University of Wisconsin - Madison using the antennas installed there along with software developed there such as CSPP Geo and CSPP Leo.

<sub>Disclaimer: I am not a meteorologist/atmospheric scientist/forecaster. I am a programmer.</sub>

## Imagers with Multiple Bands

In the world of modern meteorological satellites you will typically find each orbiting satellite with multiple instruments onboard. Satellites can be categorized as either geostationary, staying in one location over the Earth at all times, or polar-orbiting (Low Earth Orbiting - LEO), orbiting around the Earth from pole to pole multiple times a day. Different orbits can provide different benefits and different challenges (higher resolution data versus  larger area covered).

Each instrument on these satellites is designed to measure certain properties of the earth, atmosphere, or space. One type of instrument that I deal with a lot are imagers. Imager instruments use multiple passive detectors to measure electromagnetic radiation that is reflected or emitted from the earth at different wavelengths. Detectors are also referred to as bands or channels when dealing with the data. Certain molecules and objects are known to reflect or absorb energy at specific wavelengths and at specific magnitudes, this is known as the object's [spectral response pattern](https://www.e-education.psu.edu/natureofgeoinfo/c8_p5.html).

<img src="/images/water_soil_grass.gif">

So depending on which band(s) we are looking at we can determine (or take an educated guess at) what the instrument is seeing. If the instrument includes bands at or near the wavelengths of human vision we can use these bands to recreate what a human would see from space, but I'll show more on that later.

The code below shows the 16 channels available on the geostationary GOES-16 (aka GOES-EAST) ABI instrument. Note how the different bands, although all measured at the same time, show very different images because of what wavelength they represent.

In [2]:
from satpy import Scene
from glob import glob
# Load and analyze input files using the ABI L1B reader
scn = Scene(reader='abi_l1b', filenames=glob('/data/data/abi/20180711/OR_ABI-L1b-RadF-M3C??_G16_s20181921800*.nc'))
# Load all 16 channels made available from the provided files
scn.load(['C{:02d}'.format(i) for i in range(1, 17)])
# Save 8-bit GeoTIFF files in the images directory
scn.save_datasets(base_dir='../../images/')

# Use the bash command 'montage' provided by the ImageMagick tool suite
! montage -quiet ../../images/C??_20180711_180044.tif -geometry 512x512+4+4  -background black ../../images/montage_abi_20180711_180044.png

<img src="/images/montage_abi_20180711_180044.png" alt="ABI Montage" style="width:600px;">

## Projections and Resampling

There are many ways to model and show the Earth. The challenge for most people looking at a 2D display (like a computer monitor) is how to most accurately represent the Earth while still showing shapes and distances in an easy to understand way. This representation is arguably what of the most complex parts of working with satellite imagery data. Making it even more difficult to analyze data is the fact that the Earth is not a perfect sphere or even a perfect ellipsoid. Some portions of the earth are lower than others, some higher, and it isn't consistent. Over the years people have realized that it would just be easier to represent the Earth as a shape that is easy to represent mathematically. This is why you'll often see the Earth modeled as a sphere or an [ellipsoid](https://en.wikipedia.org/wiki/Spheroid) (a "smooshed" sphere) instead of a complex shape with many peaks and valleys.

Although an ellipsoid is a simpler model of the Earth, this still does not help visualizing the Earth on a 2D surface. You may want to see the whole globe at once without having to rotate it. You may want to do distance calculations or focus on a small region where dealing with a 3D representation of the Earth would be too complicated. For satellite data, you may only be seeing a portion of the Earth so representing the whole thing would serve no purpose and only make calculations more difficult. This leads us to the popular solution of projecting the Earth on to a shape that can approximate the surface of the Earth but on a 2D plane. Wikipedia can do a much better job explaining [map projections](https://en.wikipedia.org/wiki/Map_projection) and showing the many [types of projections](https://en.wikipedia.org/wiki/List_of_map_projections) used so far. Another good, although very detailed, read on maps and gridding is this chapter [here](https://www.globalsecurity.org/military/library/policy/usmc/mcwp/3-16-7/ch4.pdf). In programming, a lot of people tend to use a version of the PROJ (PROJ.4) library to handling the transformation from the spherical longitude/latitude coordinates to the X/Y coordinates of a 2D projection.

Going back to the ABI data shown in the previous section, it should be noted that this data is actually projected. The data is measured by the instrument and gridded to a projected coordinate system. In the case of ABI this is a [geos](https://proj4.org/operations/projections/geos.html) projection. It represents what the satellite instrument is actually seeing from its position above the Earth. It is as if a flat disc was placed on the Earth.

The code below takes a single channel from the example above and represents it on 2 other projections: [mercator](https://proj4.org/operations/projections/merc.html) projection and [mollweide](https://proj4.org/operations/projections/moll.html) projection.

In [ ]:
from satpy import Scene
from glob import glob
from pyresample.geometry import AreaDefinition
scn = Scene(reader='abi_l1b', filenames=glob('/data/data/abi/20180711/OR_ABI-L1b-RadF-M3C??_G16_s20181921800*.nc'))
scn.load(['C05'])

In [9]:
# resample and save to mercator projection
# make the projection 1000 pixels wide by 1000 pixels high
# make our output image start at (-8000000 meters, -5000000 meters) in the lower-left corner
# and (3000000 meters, 5000000 meters) in the upper-right corner
area_def = AreaDefinition('merc', 'merc', 'merc',
                         {'proj': 'merc'},
                         1000, 1000, (-8000000, -5000000, 3000000, 5000000))
new_scn = scn.resample(area_def)
new_scn.save_datasets(base_dir='../../images', filename='{name}_{start_time:%Y%m%d_%H%M%S}_{area.area_id}.tif')

In [4]:
# resample and save to molleweide projection
new_scn = scn.resample('moll')
new_scn.save_datasets(base_dir='../../images', filename='{name}_{start_time:%Y%m%d_%H%M%S}_{area.area_id}.tif')

In [11]:
! montage -quiet ../../images/C05_20180711_180044{,_merc,_moll}.tif -geometry 512x512+3  -background black ../../images/projections_abi_20180711_180044.png

Below we see the three projections we've discussed so far, the `geos` projection, the `mercator` projection, and the `mollweide` projection. Depending on how you define the "grid" that your data is projected to you can get different results and distortions. The key here is that each image is made from the same starting data.

<img src="/images/projections_abi_20180711_180044.png" alt="ABI Projections" style="width:600px;">

## Enhancements and Colormaps



## RGB Composites and Corrections
